In [72]:
import pandas as pd
import sqlite3

In [73]:
providers = pd.read_csv(r"D:\Guvi Projects\providers_data.csv")
receivers = pd.read_csv(r"D:\Guvi Projects\receivers_data.csv")
food_listing = pd.read_csv(r"D:\Guvi Projects\food_listings_data.csv")
claims = pd.read_csv(r"D:\Guvi Projects\claims_data.csv")

In [74]:
claims.head(2)

,Claim_ID,Food_ID,Receiver_ID,Status,Timestamp
0,1,164,908,Pending,3/5/2025 5:26
1,2,353,391,Cancelled,3/11/2025 10:24


In [75]:
providers.head(3)

,Provider_ID,Name,Type,Address,City,Contact
0,1,Gonzales-Cochran,Supermarket,"74347 Christopher Extensions\nAndreamouth, OK ...",New Jessica,+1-600-220-0480
1,2,"Nielsen, Johnson and Fuller",Grocery Store,"91228 Hanson Stream\nWelchtown, OR 27136",East Sheena,+1-925-283-8901x6297
2,3,Miller-Black,Supermarket,"561 Martinez Point Suite 507\nGuzmanchester, W...",Lake Jesusview,001-517-295-2206


In [76]:
connection = sqlite3.connect("fwms.db")
cursor = connection.cursor()

print("fwms is created")

fwms is created


In [77]:
cursor.execute("""
    CREATE TABLE IF NOT EXISTS providers (
        Provider_ID INTEGER PRIMARY KEY,
        Name TEXT,
        Type TEXT,
        Address TEXT,
        City TEXT,
        Contact TEXT
    )
""")

In [78]:
for index, row in providers[['Provider_ID', 'Name', 'Type', 'Address', 'City', 'Contact']].iterrows():
    cursor.execute("""
        INSERT OR IGNORE INTO providers (Provider_ID, Name, Type, Address, City, Contact) 
        VALUES (?, ?, ?, ?, ?, ?)
    """, tuple(row))

In [89]:
receivers.tail(2)

,Receiver_ID,Name,Type,City,Contact
998,999,Cheyenne Ramsey,NGO,Lake Jeffery,001-326-320-4816x15300
999,1000,April Miller,Shelter,Shelbychester,894-873-2652x23999


In [80]:
cursor.execute("""
    CREATE TABLE IF NOT EXISTS receivers (
            Receiver_ID INTEGER PRIMARY KEY,
            Name TEXT,
            Type TEXT,
            City TEXT,
            Contact TEXT
    )
""")

In [81]:
for index, row in receivers[['Receiver_ID', 'Name', 'Type', 'City', 'Contact']].iterrows():
    cursor.execute("""
        INSERT OR IGNORE INTO receivers (Receiver_ID, Name, Type, City, Contact) 
        VALUES (?, ?, ?, ?, ?)
    """, tuple(row))


In [82]:
food_listing.head(2)

,Food_ID,Food_Name,Quantity,Expiry_Date,Provider_ID,Provider_Type,Location,Food_Type,Meal_Type
0,1,Bread,43,3/17/2025,110,Grocery Store,South Kellyville,Non-Vegetarian,Breakfast
1,2,Soup,22,3/24/2025,791,Grocery Store,West James,Non-Vegetarian,Dinner


In [83]:
cursor.execute("""
    CREATE TABLE IF NOT EXISTS food_listing (
        Food_ID INTEGER PRIMARY KEY,
        Food_Name TEXT,
        Quantity INTEGER,
        Expiry_Date TEXT,
        Provider_ID INTEGER,
        Provider_Type TEXT,
        Location TEXT,
        Food_Type TEXT,
        Meal_Type TEXT
    )
""")

In [86]:
for index, row in food_listing.iterrows():
    cursor.execute("DELETE FROM food_listing WHERE Food_ID = ?", (row['Food_ID'],))
    cursor.execute("""
        INSERT INTO food_listing (Food_ID, Food_Name, Quantity, Expiry_Date, Provider_ID, Provider_Type, Location, Food_Type, Meal_Type)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
    """, tuple(row))

connection.commit()

In [ ]:
claims.head(2)

,Claim_ID,Food_ID,Receiver_ID,Status,Timestamp
0,1,164,908,Pending,3/5/2025 5:26
1,2,353,391,Cancelled,3/11/2025 10:24


In [ ]:
cursor.execute("""
    CREATE TABLE IF NOT EXISTS claims (
        Claim_ID INTEGER PRIMARY KEY,
        Food_ID INTEGER,
        Receiver_ID INTEGER,
        Status TEXT,
        Timestamp TEXT,
        FOREIGN KEY (Food_ID) REFERENCES food_listing(Food_ID),
        FOREIGN KEY (Receiver_ID) REFERENCES receivers(Receiver_ID)
    )
""")

In [ ]:
for index, row in claims[['Food_ID', 'Receiver_ID', 'Status', 'Timestamp']].iterrows():
    cursor.execute("""
        INSERT INTO claims (Food_ID, Receiver_ID, Status, Timestamp)
        VALUES (?, ?, ?, ?)
    """, tuple(row)) 

In [ ]:
connection.commit()

1.How many food providers and receivers are there in each city?

In [93]:
query_1 = """
    SELECT 
        p.City AS City, 
        COUNT(DISTINCT p.Provider_ID) AS Number_of_Providers, 
        COUNT(DISTINCT r.Receiver_ID) AS Number_of_Receivers
    FROM 
        providers p
    LEFT JOIN 
        receivers r ON p.City = r.City
    GROUP BY 
        p.City;
"""
cursor.execute(query_1)
results = cursor.fetchall()
print("City-wise count of Providers and Receivers:")
for row in results:
    print(f"City: {row[0]}, Providers: {row[1]}, Receivers: {row[2]}")


City-wise count of Providers and Receivers:
City: Adambury, Providers: 1, Receivers: 0
City: Adamsview, Providers: 1, Receivers: 0
City: Adamsville, Providers: 1, Receivers: 0
City: Aguirreville, Providers: 1, Receivers: 0
City: Alexanderchester, Providers: 1, Receivers: 0
City: Alexanderstad, Providers: 1, Receivers: 0
City: Allenborough, Providers: 1, Receivers: 1
City: Allenton, Providers: 1, Receivers: 0
City: Amandaborough, Providers: 1, Receivers: 0
City: Amandashire, Providers: 1, Receivers: 0
City: Amberton, Providers: 1, Receivers: 0
City: Ambertown, Providers: 1, Receivers: 0
City: Amyport, Providers: 1, Receivers: 0
City: Andersonmouth, Providers: 1, Receivers: 0
City: Andersonville, Providers: 1, Receivers: 0
City: Andreaborough, Providers: 1, Receivers: 0
City: Andrewsmouth, Providers: 1, Receivers: 0
City: Andrewstad, Providers: 1, Receivers: 0
City: Anitashire, Providers: 1, Receivers: 0
City: Annahaven, Providers: 1, Receivers: 0
City: Annetteburgh, Providers: 1, Receiv

2.Which type of food provider (restaurant, grocery store, etc.) contributes the most food?


In [94]:
query_2 = """
    SELECT Provider_Type, SUM(Quantity) AS Total_Quantity
    FROM food_listing
    GROUP BY Provider_Type
    ORDER BY Total_Quantity DESC
    LIMIT 1;
"""
cursor.execute(query_2)
result = cursor.fetchone()
if result:
    print(f"The provider type that contributes the most food is '{result[0]}' with a total of {result[1]} units.")

The provider type that contributes the most food is 'Restaurant' with a total of 6923 units.


3.What is the contact information of food providers in a specific city?

In [ ]:
providers.head(2)

,Provider_ID,Name,Type,Address,City,Contact
0,1,Gonzales-Cochran,Supermarket,"74347 Christopher Extensions\nAndreamouth, OK ...",New Jessica,+1-600-220-0480
1,2,"Nielsen, Johnson and Fuller",Grocery Store,"91228 Hanson Stream\nWelchtown, OR 27136",East Sheena,+1-925-283-8901x6297


In [95]:
specific_city = input()
query_3 = """
    SELECT Name, Contact
    FROM Providers
    WHERE City = ?;
    """ 
cursor.execute(query_3,(specific_city,))

result = cursor.fetchall()
print(f"Contact information of providers in {specific_city}:")
for row in result:
    print(f"Name: {row[0]}, Contact: {row[1]}")


Contact information of providers in :


4.Which receivers have claimed the most food?

In [ ]:
claims.head(2)

,Claim_ID,Food_ID,Receiver_ID,Status,Timestamp
0,1,164,908,Pending,3/5/2025 5:26
1,2,353,391,Cancelled,3/11/2025 10:24


In [96]:
query_4 = """ 
    SELECT r.Name, r.Receiver_ID, COUNT(c.claim_ID) AS Total_Claims
    FROM claims c
    JOIN receivers r ON c.Receiver_ID = r.Receiver_ID
    GROUP BY c.Receiver_ID
    ORDER BY Total_Claims DESC
    LIMIT 1;
    """

cursor.execute(query_4)
result = cursor.fetchone()
print(f"The most food claimed receiver is '{result[0]}'")

The most food claimed receiver is 'Anthony Garcia'


5.What is the total quantity of food available from all providers?


In [ ]:
food_listing.head(2)

,Food_ID,Food_Name,Quantity,Expiry_Date,Provider_ID,Provider_Type,Location,Food_Type,Meal_Type
0,1,Bread,43,3/17/2025,110,Grocery Store,South Kellyville,Non-Vegetarian,Breakfast
1,2,Soup,22,3/24/2025,791,Grocery Store,West James,Non-Vegetarian,Dinner


In [97]:
query_5 = """
    SELECT SUM(Quantity) AS Total_Quantity
    FROM food_listing;
    """
cursor.execute(query_5)
result = cursor.fetchone()
print(f"Total quantity of food available from all providers: {result[0]}")

Total quantity of food available from all providers: 26276


6.Which city has the highest number of food listings?


In [98]:
query_6 = """
    SELECT Location, COUNT(*) AS Listings_Count
    FROM food_listing
    GROUP BY Location
    ORDER BY Listings_Count DESC
    LIMIT 1;
"""
cursor.execute(query_6)

result = cursor.fetchone()
print(f"City with the highest number of food listings: {result[0]}")

City with the highest number of food listings: South Kathryn


7.What are the most commonly available food types?

In [99]:
query_7 = """
    SELECT Food_Type, COUNT(*) AS Count
    FROM food_listing
    GROUP BY Food_Type
    ORDER BY Count DESC;
"""
cursor.execute(query_7)

results = cursor.fetchall()

print("Most commonly available food types are:")
for row in results:
    print(f"{row[0]} with {row[1]} listings")

Most commonly available food types are:
Vegetarian with 336 listings
Vegan with 334 listings
Non-Vegetarian with 330 listings


8.How many food claims have been made for each food item?


In [100]:
query_8 = """
     SELECT  f.Food_Name, 
        f.Food_ID, 
        COUNT(c.Claim_ID) AS Total_Claims
    FROM 
        claims c
    JOIN 
        food_listing f ON c.Food_ID = f.Food_ID
    GROUP BY 
        c.Food_ID
    ORDER BY 
        Total_Claims DESC;
"""
cursor.execute(query_8)

results = cursor.fetchall()


print("Number of food claims per food item:")
for row in results:
    print(f"Food: {row[0]} (ID: {row[1]}) - {row[2]} claims")

Number of food claims per food item:
Food: Fish (ID: 548) - 5 claims
Food: Chicken (ID: 486) - 5 claims
Food: Soup (ID: 463) - 5 claims
Food: Dairy (ID: 981) - 4 claims
Food: Salad (ID: 925) - 4 claims
Food: Vegetables (ID: 915) - 4 claims
Food: Bread (ID: 886) - 4 claims
Food: Salad (ID: 882) - 4 claims
Food: Dairy (ID: 861) - 4 claims
Food: Fish (ID: 832) - 4 claims
Food: Dairy (ID: 797) - 4 claims
Food: Rice (ID: 696) - 4 claims
Food: Vegetables (ID: 674) - 4 claims
Food: Fish (ID: 520) - 4 claims
Food: Dairy (ID: 251) - 4 claims
Food: Soup (ID: 217) - 4 claims
Food: Salad (ID: 190) - 4 claims
Food: Chicken (ID: 92) - 4 claims
Food: Rice (ID: 35) - 4 claims
Food: Rice (ID: 964) - 3 claims
Food: Pasta (ID: 952) - 3 claims
Food: Bread (ID: 947) - 3 claims
Food: Vegetables (ID: 945) - 3 claims
Food: Bread (ID: 937) - 3 claims
Food: Dairy (ID: 912) - 3 claims
Food: Fish (ID: 909) - 3 claims
Food: Fish (ID: 870) - 3 claims
Food: Soup (ID: 837) - 3 claims
Food: Dairy (ID: 822) - 3 claims


9.Which provider has had the highest number of successful food claims?


In [ ]:
claims.head(3)

,Claim_ID,Food_ID,Receiver_ID,Status,Timestamp
0,1,164,908,Pending,3/5/2025 5:26
1,2,353,391,Cancelled,3/11/2025 10:24
2,3,626,492,Completed,3/21/2025 0:59


In [ ]:
providers.head(2)

,Provider_ID,Name,Type,Address,City,Contact
0,1,Gonzales-Cochran,Supermarket,"74347 Christopher Extensions\nAndreamouth, OK ...",New Jessica,+1-600-220-0480
1,2,"Nielsen, Johnson and Fuller",Grocery Store,"91228 Hanson Stream\nWelchtown, OR 27136",East Sheena,+1-925-283-8901x6297


In [ ]:
food_listing.head(2)

,Food_ID,Food_Name,Quantity,Expiry_Date,Provider_ID,Provider_Type,Location,Food_Type,Meal_Type
0,1,Bread,43,3/17/2025,110,Grocery Store,South Kellyville,Non-Vegetarian,Breakfast
1,2,Soup,22,3/24/2025,791,Grocery Store,West James,Non-Vegetarian,Dinner


In [105]:
query_9 = """ 
    SELECT 
        p.Name AS Provider_Name,
        p.Provider_ID,
        COUNT(c.Claim_ID) AS Successful_Claims
        FROM claims C
        JOIN food_listing f ON c.Food_ID = f.Food_ID
        JOIN providers p ON f.Provider_ID = p.Provider_ID
        WHERE c.Status = 'Completed'
        GROUP BY p.Provider_ID
        ORDER BY Successful_claims DESC
        LIMIT 1;
        """
cursor.execute(query_9)
result = cursor.fetchone()
print(f"'{result[0]}' has had the highest number of food claims")

'Barry Group' has had the highest number of food claims


10.What percentage of food claims are completed vs. pending vs. canceled?


In [102]:
query_10 =""" 
    SELECT Status,
    COUNT(*) * 100.0 / (SELECT COUNT(*) FROM claims) AS Percentage
    FROM claims 
    GROUP BY Status;
"""
cursor.execute(query_10)

result = cursor.fetchall()

for status, percentage in result:
    print(f"{status} = {percentage:.2f}%")

Cancelled = 33.60%
Completed = 33.90%
Pending = 32.50%


11.What is the average quantity of food claimed per receiver?


In [104]:
query_11 = """
    SELECT c.Receiver_ID, r.Name,
    AVG(f.Quantity) AS Avg_Food_Claimed_Quantity
    FROM claims c
    JOIN food_listing f ON c.Food_ID = f.Food_ID
    JOIN receivers r ON c.Receiver_ID = r.Receiver_ID
    GROUP BY c.Receiver_ID
    ORDER BY Avg_Food_Claimed_Quantity DESC; """         
cursor.execute(query_11)
result = cursor.fetchall()
for Receiver_ID, Name, Avg_Food_Claimed_Quantity in result:
    print(f"{Name} with ID {Receiver_ID}- Avg_Food_Claimed_Quantity = {Avg_Food_Claimed_Quantity:.2f}")

Jean Woods with ID 887- Avg_Food_Claimed_Quantity = 287.50
Thomas Villanueva with ID 982- Avg_Food_Claimed_Quantity = 50.00
Nancy Jones with ID 739- Avg_Food_Claimed_Quantity = 50.00
Christopher Wright with ID 616- Avg_Food_Claimed_Quantity = 50.00
Peggy Knight with ID 567- Avg_Food_Claimed_Quantity = 50.00
Daniel Williams with ID 429- Avg_Food_Claimed_Quantity = 50.00
Lisa Pitts with ID 282- Avg_Food_Claimed_Quantity = 50.00
Nancy Silva with ID 99- Avg_Food_Claimed_Quantity = 50.00
Christopher Gonzalez with ID 906- Avg_Food_Claimed_Quantity = 49.00
Cassandra Yoder with ID 772- Avg_Food_Claimed_Quantity = 49.00
Jennifer Mccoy with ID 408- Avg_Food_Claimed_Quantity = 49.00
Angela Wyatt with ID 279- Avg_Food_Claimed_Quantity = 49.00
Melissa Little with ID 100- Avg_Food_Claimed_Quantity = 49.00
Timothy Patel DVM with ID 94- Avg_Food_Claimed_Quantity = 49.00
Scott Brown with ID 69- Avg_Food_Claimed_Quantity = 49.00
Alexander Mullins with ID 38- Avg_Food_Claimed_Quantity = 49.00
Angel Adams

12. Which meal type (breakfast, lunch, dinner, snacks) is claimed the most?


In [125]:
query_12 = """
    SELECT f.Meal_Type,
    COUNT(*) AS Total_Claims
    FROM claims c
    JOIN food_listing f ON c.Food_ID = f.Food_ID
    GROUP BY f.Meal_Type
    ORDER BY Total_Claims DESC
    LIMIT 1; """
cursor.execute(query_12)
result = cursor.fetchone()
Meal_Type = result
print(f"'{result[0]}' is claimed the most")

'Breakfast' is claimed the most


13.What is the total quantity of food donated by each provider?


In [110]:
query_13 = """
    SELECT 
    p.Provider_ID, p.Name,
    SUM(f.Quantity) AS Total_Donated
    FROM food_listing f
    JOIN providers p ON f.Provider_ID = p.Provider_ID
    GROUP BY p.Provider_ID
    ORDER BY Total_Donated DESC; """      
cursor.execute(query_13)
result = cursor.fetchall()
for Provider_ID, Name, Total in result:
    print(f"{Name} with ID {Provider_ID} = Total Donated {Total}")

Lopez, Roach and Roach with ID 478 = Total Donated 584
Barry Group with ID 709 = Total Donated 179
Evans, Wright and Mitchell with ID 306 = Total Donated 158
Smith Group with ID 655 = Total Donated 150
Nelson LLC with ID 315 = Total Donated 142
Ruiz-Oneal with ID 678 = Total Donated 140
Blankenship-Lewis with ID 499 = Total Donated 124
Campbell LLC with ID 161 = Total Donated 123
Kelly-Ware with ID 41 = Total Donated 123
Bradford-Martinez with ID 262 = Total Donated 121
Shepherd and Sons with ID 717 = Total Donated 116
Hampton-Lee with ID 146 = Total Donated 116
Jones, Ortega and Rubio with ID 498 = Total Donated 115
Ortiz-Lee with ID 7 = Total Donated 114
Johnson-Ray with ID 139 = Total Donated 113
Miller Inc with ID 846 = Total Donated 111
Barker LLC with ID 938 = Total Donated 110
Moore Group with ID 995 = Total Donated 106
Hill, Davis and Stewart with ID 518 = Total Donated 106
Butler-Richardson with ID 752 = Total Donated 104
Steele Ltd with ID 393 = Total Donated 104
Carter-Jones

Additional Questions

14.Which provider has had the highest number of failured food claims?


In [111]:
query_14 = """ 
    SELECT 
        p.Name AS Provider_Name,
        p.Provider_ID,
        COUNT(c.Claim_ID) AS Unsuccessful_Claims
        FROM claims C
        JOIN food_listing f ON c.Food_ID = f.Food_ID
        JOIN providers p ON f.Provider_ID = p.Provider_ID
        WHERE c.Status = 'Cancelled'
        GROUP BY p.Provider_ID
        ORDER BY Unsuccessful_claims DESC
        LIMIT 1;
        """
cursor.execute(query_14)
result = cursor.fetchone()
print(f"Provider '{result[0]}' has had the highest number of failured food claims")

Provider 'Butler-Richardson' has had the highest number of failured food claims


15.What are all the Foods Grocery can supply?

In [112]:
query_15 = """
    SELECT DISTINCT f.Food_Name
    FROM food_listing f
    WHERE f.Provider_Type = "Grocery Store"; """
cursor.execute(query_15)
result = cursor.fetchall()
print("Foods supplied by grocery")
for (Food_Name,) in result:
    print(Food_Name)

Foods supplied by grocery
Bread
Soup
Dairy
Pasta
Fruits
Chicken
Salad
Fish
Rice
Vegetables


In [ ]:
receivers.head(3)

,Receiver_ID,Name,Type,City,Contact
0,1,Donald Gomez,Shelter,Port Carlburgh,(955)922-5295
1,2,Laurie Ramos,Individual,Lewisburgh,761.042.1570
2,3,Ashley Mckee,NGO,South Randalltown,691-023-0094x856


16.What is the total quantity of food available in each city?

In [113]:
query_16 = """
    SELECT f.Location AS City,
    SUM(f.Quantity) AS Total_Quantity
    FROM food_listing f
    GROUP BY f.Location
    ORDER BY Total_Quantity DESC; """      
cursor.execute(query_16)
result = cursor.fetchall()
for City, Quantity in result:
    print(f"{City} has availability of {Quantity} Quantity")

coimbatore has availability of 528 Quantity
South Kathryn has availability of 179 Quantity
Jonathanstad has availability of 169 Quantity
New Carol has availability of 167 Quantity
North Keith has availability of 158 Quantity
Jimmyberg has availability of 150 Quantity
Lake Andrewmouth has availability of 142 Quantity
Lake Travis has availability of 140 Quantity
South Christopherborough has availability of 137 Quantity
Port Patrick has availability of 127 Quantity
Perezport has availability of 123 Quantity
Bentleyburgh has availability of 123 Quantity
Bradleyport has availability of 120 Quantity
New Amanda has availability of 119 Quantity
Pamelaberg has availability of 116 Quantity
Jonathanhaven has availability of 116 Quantity
North Kevinhaven has availability of 115 Quantity
East Craig has availability of 115 Quantity
Lake Christopherburgh has availability of 114 Quantity
Port Connie has availability of 113 Quantity
South Thomasville has availability of 111 Quantity
Hollyhaven has avai

17.How many pending claims are there for each city?

In [114]:
query_17 = """
    SELECT f.Location AS City,
    COUNT(c.Claim_ID) AS Pending_claims
    FROM claims c
    JOIN food_listing f ON c.Food_ID = f.Food_ID
    WHERE Status = 'Pending'
    GROUP BY f.Location
    ORDER BY Pending_claims DESC; """            
cursor.execute(query_17)
result = cursor.fetchall()
for City, Pending_claims in result:
    print(f"{City} has pending claims of {Pending_claims}")

North Kevinhaven has pending claims of 4
Lake Andrewmouth has pending claims of 4
West Billborough has pending claims of 3
Watsonstad has pending claims of 3
Thomasville has pending claims of 3
North William has pending claims of 3
North Ianbury has pending claims of 3
New Curtis has pending claims of 3
Muellermouth has pending claims of 3
Lake Stephen has pending claims of 3
Lake Mistyton has pending claims of 3
Jamesfurt has pending claims of 3
Frederickside has pending claims of 3
East Stephanie has pending claims of 3
East John has pending claims of 3
East Austin has pending claims of 3
Allenton has pending claims of 3
Williamsmouth has pending claims of 2
West Sharonview has pending claims of 2
West Karen has pending claims of 2
Stevenchester has pending claims of 2
Stephanieberg has pending claims of 2
South Thomasville has pending claims of 2
South Randy has pending claims of 2
South Kellyberg has pending claims of 2
Reidland has pending claims of 2
Priceland has pending claims 

18.What types of food are supplied by each provider?

In [ ]:
providers.head(1)

,Provider_ID,Name,Type,Address,City,Contact
0,1,Gonzales-Cochran,Supermarket,"74347 Christopher Extensions\nAndreamouth, OK ...",New Jessica,+1-600-220-0480


In [ ]:
food_listing.head(1)

,Food_ID,Food_Name,Quantity,Expiry_Date,Provider_ID,Provider_Type,Location,Food_Type,Meal_Type
0,1,Bread,43,3/17/2025,110,Grocery Store,South Kellyville,Non-Vegetarian,Breakfast


In [115]:
query_18 = """ 
    SELECT DISTINCT f.Food_Type AS Type,
    p.Name AS Provider_name
    FROM food_listing f
    JOIN providers p ON p.Provider_ID = f.Provider_ID
    ORDER BY p.Provider_ID DESC; """
cursor.execute(query_18)
result = cursor.fetchall()
for Type, Provider_name in result:
    print(f"{Provider_name} supplies '{Type}' type of Food")

Jenkins-Brooks supplies 'Vegan' type of Food
Moody Ltd supplies 'Non-Vegetarian' type of Food
Moody Ltd supplies 'Vegan' type of Food
Novak Group supplies 'Vegan' type of Food
Garza-Williams supplies 'Vegan' type of Food
Moore Group supplies 'Vegan' type of Food
Moore Group supplies 'Vegetarian' type of Food
Mckinney Ltd supplies 'Non-Vegetarian' type of Food
Mckinney Ltd supplies 'Vegetarian' type of Food
Mckinney Ltd supplies 'Vegan' type of Food
Jones PLC supplies 'Vegetarian' type of Food
Williams-Aguirre supplies 'Non-Vegetarian' type of Food
Harris, Santos and Roberts supplies 'Non-Vegetarian' type of Food
Wilson, Harris and Mayo supplies 'Vegan' type of Food
Wilson, Harris and Mayo supplies 'Non-Vegetarian' type of Food
Frey PLC supplies 'Non-Vegetarian' type of Food
Gamble-Davis supplies 'Vegetarian' type of Food
Munoz, Stone and Wallace supplies 'Vegan' type of Food
Munoz, Stone and Wallace supplies 'Non-Vegetarian' type of Food
Martinez PLC supplies 'Vegan' type of Food
Glenn

19.Which receivers have claimed both Vegetarian and Non-Vegetarian food?

In [ ]:
receivers.head(1)

,Receiver_ID,Name,Type,City,Contact
0,1,Donald Gomez,Shelter,Port Carlburgh,(955)922-5295


In [116]:
claims.head(2)

,Claim_ID,Food_ID,Receiver_ID,Status,Timestamp
0,1,164,908,Pending,3/5/2025 5:26
1,2,353,391,Cancelled,3/11/2025 10:24


In [117]:
food_listing.head(5)

,Food_ID,Food_Name,Quantity,Expiry_Date,Provider_ID,Provider_Type,Location,Food_Type,Meal_Type
0,1,Bread,43,3/17/2025,110,Grocery Store,South Kellyville,Non-Vegetarian,Breakfast
1,2,Soup,22,3/24/2025,791,Grocery Store,West James,Non-Vegetarian,Dinner
2,3,Fruits,46,3/28/2025,478,Catering Service,Lake Regina,Vegan,Breakfast
3,4,Fruits,15,3/16/2025,930,Restaurant,Kellytown,Vegan,Lunch
4,5,Soup,14,3/19/2025,279,Restaurant,Garciaport,Vegan,Dinner


In [118]:
query_19 = """ 
    SELECT c.Receiver_ID, r.Name
    FROM claims c
        JOIN receivers r ON c.Receiver_ID = r.Receiver_ID
        JOIN food_listing f ON c.Food_ID = f.Food_ID
    GROUP BY c.Receiver_ID, r.Name
    HAVING
        SUM(CASE WHEN f.Food_Type = 'Vegetarian' THEN 1 ELSE 0 END) >0
        AND
        SUM(CASE WHEN f.Food_Type = 'Non-Vegetarian' THEN 1 ELSE 0 END) >0;
    """     
cursor.execute(query_19)
result = cursor.fetchall()
print("Receivers who claimes both Veg and Non-Veg are: ")
for Receiver_ID, Name in result:
    print(f"{Name} with ID {Receiver_ID}")

Receivers who claimes both Veg and Non-Veg are: 
Mandy Sutton PhD with ID 6
Mary Franklin with ID 28
Wesley Thompson with ID 68
Jessica Grant with ID 86
Allison Bray with ID 93
Michael Walls with ID 96
Daniel Burton with ID 102
Jonathan Young with ID 113
Matthew Tran with ID 118
Heather Brown with ID 126
Sarah Carter with ID 128
Katherine Gomez with ID 132
Jennifer Nelson with ID 144
Nicole Brown with ID 172
Nicole Walker with ID 183
Rhonda King with ID 192
Morgan Massey with ID 216
Alicia Porter with ID 233
Rebecca Sanchez with ID 245
Derek Wilson with ID 256
Aimee Stone with ID 259
Autumn Reed with ID 262
Scott Hunter with ID 276
Derrick Curtis with ID 285
Cynthia Horton with ID 294
Kristina Simpson with ID 301
Linda Banks with ID 323
Anna Buckley with ID 341
Donna Hughes with ID 348
Kimberly Nelson with ID 354
Samuel Lewis with ID 361
Debra Murray with ID 368
William Frederick with ID 371
Patrick Campbell with ID 377
Victor Crawford with ID 390
Marissa Fischer with ID 404
Arthur Tho

20.Which receiver has claimed the most Breakfast items?

In [120]:
query_20 = """ 
    SELECT c.Receiver_ID, r.Name,
    COUNT(c.Claim_ID) AS Breakfast_claims
    FROM claims c
        JOIN food_listing f ON c.Food_ID = f.Food_ID
        JOIN receivers r ON c.Receiver_ID = r.Receiver_ID
    WHERE f.Meal_Type = 'Breakfast'
    GROUP BY c.Receiver_ID, r.Name
    ORDER BY Breakfast_claims DESC
    LIMIT 1; """           
cursor.execute(query_20)
result = cursor.fetchone()
print(f"'{result[1]}' has claimed the most Breakfast")

'Kristina Simpson' has claimed the most Breakfast


21.What are the top 5 most commonly claimed food items across all receivers?

In [121]:
query_21 = """
    SELECT f.Food_Name, 
    COUNT(c.Claim_ID) AS Claim_Count
    FROM claims c
        JOIN food_listing f ON c.Food_ID = f.Food_ID
    GROUP BY 
        f.Food_ID, f.Food_Name
    ORDER BY 
        Claim_Count DESC
    LIMIT 5;
"""
cursor.execute(query_21)
result = cursor.fetchall()
print("5 most commomly claimed food items are")
for row in result:
    print(f"{row[0]}- {row[1]} claims")


5 most commomly claimed food items are
Soup- 5 claims
Chicken- 5 claims
Fish- 5 claims
Rice- 4 claims
Chicken- 4 claims


In [ ]:
receivers.head(15)

,Receiver_ID,Name,Type,City,Contact
0,1,Donald Gomez,Shelter,Port Carlburgh,(955)922-5295
1,2,Laurie Ramos,Individual,Lewisburgh,761.042.1570
2,3,Ashley Mckee,NGO,South Randalltown,691-023-0094x856
3,4,Erika Rose,NGO,South Shaneville,8296491111
4,5,John Romero,Individual,Bakerport,067.491.0154
5,6,Mandy Sutton PhD,Shelter,East Sharimouth,682-777-5357
6,7,Kenneth Baker,NGO,South Edwinborough,126-645-6386x4071
7,8,James Perez,Charity,Benjaminburgh,+1-986-748-6688
8,9,Emily Turner,Shelter,West Robert,001-631-022-6157x520
9,10,Mary Salazar,NGO,Wrightland,001-187-333-7518x11395


22.Which type of receiver is more in quantity?

In [122]:
query_22 = """ 
    SELECT r.Type AS Receiver_Type,
    COUNT(r.Receiver_ID) AS Receiver_Count
    FROM receivers r
    GROUP BY r.Type
    ORDER BY Receiver_Count DESC
    LIMIT 1; """           
cursor.execute(query_22)
result = cursor.fetchone()
print(f"The highest receiver type is {result}")

The highest receiver type is ('NGO', 274)


23.What is the total quantity of food that is claimed by NGO?

In [123]:
query_23 = """
    SELECT 
        SUM(f.Quantity) AS Total_Quantity_Claimed
    FROM 
        claims c
        JOIN food_listing f ON c.Food_ID = f.Food_ID
        JOIN receivers r ON c.Receiver_ID = r.Receiver_ID
    WHERE 
        r.Type = 'NGO';
"""
cursor.execute(query_23)
result = cursor.fetchone()
print(f"Total quantity of food claimed by NGO is {result[0]}")

Total quantity of food claimed by NGO is 6910


In [126]:
connection.commit()
connection.close()